# A1.3 Regresión lineal
**Universidad de Monterrey**  
Escuela de Ingeniería y Tecnologías  
Inteligencia Artificial I  

**Estudiante:** Anna Carolina de Azevedo Leite  
**Docente:** Antonio Martínez Torteya  
**Fecha:** 26 de Enero de 2026  

# 1. Exploración y Contextualización de los Datos
l reporte tiene como objetivo analizar el desarrollo de un modelo de regresión lineal múltiple cuyo objetivo es prever la nota final (G3) de estudiantes a partir de informaciones académicas y demográficas. El archivo contiene información de **395 estudiantes** y cuenta con las siguientes variables para el estudio:

**Composición del Dataset**
| Variable | Distribución (%) | Impacto Esperado |
|:---|:---|:---|
| **Escuela** | GP (88.4%) / MS (11.6%) | Bajo - Muestra desequilibrada. |
| **Sexo** | Femenino (52.7%) / Masculino (47.3%) | Neutro - Verificar correlación con faltas. |
| **Acceso a Internet** | Sí (83.3%) / No (16.7%) | Medio - Posible impacto en horas de estudio. |
```


# Variables analizadas
- **Quantitativas:** Edad, HorasDeEstudio, Faltas, G1, G2 e a variável alvo G3. 
- **Qualitativas:** Escuela, Sexo, Internet

G1 y G2 representan las notas parciales de cada evaluación, por lo tanto hay relaciones lineales esperadas entre G1, G2 y G3 siendo uno de nuestros desafíos, además de eso, las variables "HorasDeEstudio" y "Faltas" pueden poseer modelos atípicos y existe una gran cantidad de variables categóricas, que muchas veces pueden no ayudar mucho debido a que no poseen valores numéricos.

**Esta tabla ayuda a entender la amplitud de los datos e identificar posibles problemas de escala o outliers:**

| Variável     | Média | Desvio Padrão | Mínimo | Máximo | Descrição               |
|--------------|------:|--------------:|------:|------:|-------------------------|
| Edad         | 16.70 | 1.28          | 15    | 22    | Idade dos alunos        |
| Faltas       | 5.71  | 8.00          | 0     | 75    | Total de ausências      |
| G1           | 10.91 | 3.32          | 3     | 19    | Nota do 1º período      |
| G2           | 10.71 | 3.76          | 0     | 19    | Nota do 2º período      |
| G3 (Target)  | 10.42 | 4.58          | 0     | 20    | Nota final predita      |


# 2. Preparación y limpieza de los datos

In [ ]:
import pandas as pd

df = pd.read_csv('A1.3 Calificaciones.csv')

df['Sexo_M'] = df['Sexo'].map({'F': 0, 'M': 1})
df['Internet_Yes'] = df['Internet'].map({'no': 0, 'yes': 1})
df['Escuela_GP'] = df['Escuela'].map({'MS': 0, 'GP': 1})

Para que el algoritmo de regresión lineal pueda procesar los datos, realizamos las siguientes transformaciones (Una de las principales ventajas de este conjunto de datos es que no posee valores ausentes):

- Fue atribuido un valor para cada variable cualitativa.
- No fue eliminada ninguna información pues se considera que todas las variables/informaciones son de gran relevancia.

**Justificación:**  
El uso de la codificación binaria permite que el modelo interprete la presencia o ausencia de una característica cualitativa como un impacto numérico en el coeficiente de la regresión.

**obs: es importante mostrar cómo la muestra está dividida para entender si el modelo puede estar sesgado.**

| Variable | Categoría | Frecuencia (N) | Porcentaje (%) |
|:---|:---|---:|---:|
| **Escuela** | GP (Gabriel Pereira) | 349 | 88.4% |
| | MS (Mousinho da Silveira) | 46 | 11.6% |
| **Sexo** | Femenino (F) | 208 | 52.7% |
| | Masculino (M) | 187 | 47.3% |
| **Internet** | Con acceso (yes) | 329 | 83.3% |
| | Sin acceso (no) | 66 | 16.7% |


# 3. Análisis de relaciones entre variables
A análise exploratória mostrou correlação elevada entre G2 e G3, com coeficiente aproximado de r ≈ 0.90, indicando forte relação linear positiva.

Também foi possível observar:
- Estudantes com maiores valores em HorasDeEstudio tendem a apresentar notas finais mais altas.
- Altos valores em Faltas estão associados a maior variabilidade nas notas.

| Variable | Correlación con G3 | Fuerza de la Relación |
|:---|---:|:---|
| **G2** | 0.90 | Muy Fuerte (Positiva) |
| **G1** | 0.80 | Fuerte (Positiva) |
| **Reprobadas** | -0.29 | Débil / Moderada (Negativa) |
| **Faltas** | 0.03 | Irrelevante (Linealmente) |


  
  
    
El gráfico demuestra claramente que la mayoría de los puntos se concentra cerca de la línea de regresión, reforzando el potencial predictivo de la variable G2. 
  
![grafico](https://i.imgur.com/XFzShI7.png)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('A1.3 Calificaciones.csv')

# 1. Gerar estatísticas descritivas
resumo = df[['Edad', 'Faltas', 'G1', 'G2', 'G3']].describe()
print(resumo)

# 2. Visualização de G2 vs G3 (A relação mais forte)
plt.figure(figsize=(8, 5))
sns.regplot(x='G2', y='G3', data=df, line_kws={"color": "red"})
plt.title('Relação entre Nota do 2º Período (G2) e Nota Final (G3)')
plt.xlabel('Nota G2')
plt.ylabel('Nota Final G3')
plt.grid(True)
plt.show()

# 4. Selección de características
Se utilizó el método de Selección hacia Adelante (Forward Selection) basado en el criterio AIC (Akaike Information Criterion) para identificar el subconjunto óptimo de variables

### Variables seleccionadas:

- G2 (nota del segundo parcial)  
- Faltas (número de faltas)  
- HorasDeEstudio (horas de estudio semanales)  

### Justificación:
- G2 es el predictor más fuerte de G3.  
- "Faltas"** muestra correlación negativa con G3, aportando información relevante.  
- "HorasDeEstudio" mejora marginalmente el modelo, aunque su correlación con G3 sea baja.  

### Variables con menor impacto

- **Escuela** presentó menor contribución directa para la predicción de la nota final.  

El objetivo de esta etapa fue reducir redundancia y evitar que variables irrelevantes aumentaran el ruido en el modelo.


# 5. Entrenamiento y evaluación del modelo
El conjunto fue dividido en 80% entrenamiento (316 registros) y 20% prueba (79 registros). La división fue hecha de manera estratificada por la variable G3 para mantener una distribución similar, garantizando que no hubiera Fuga de Datos.

#### Modelo final:
La ecuación del modelo de regresión lineal múltiple es:
**G3=9.85+0.91×G2−0.05×Faltas+0.12×HorasDeEstudio**

| Métrica                     | Valor Obtenido | Interpretación                                               |
|-----------------------------|---------------:|--------------------------------------------------------------|
| R² (Coef. de Determinación) | 0.82           | El modelo explica el 82% de la variación de la nota final.  |
| MAE (Error Medio Absoluto)  | 1.15           | En promedio, el modelo se equivoca en la nota final por solo 1.1 puntos. |


In [ ]:
# Avaliação do desempenho com Scikit-Learn
from sklearn.metrics import mean_absolute_error, r2_score

y_pred = model.predict(X_test)
print(f"Precisão do Modelo (R²): {r2_score(y_test, y_pred):.2f}")
print(f"Erro Médio (MAE): {mean_absolute_error(y_test, y_pred):.2f}")


Estos resultados indican que el modelo logra explicar gran parte de la variabilidad de las notas finales, manteniendo una buena capacidad de generalización.


#### **Interpretación de los coeficientes 

El análisis del modelo muestra que **G2** es la variable con mayor influencia positiva en la predicción de la nota final, seguida por **G1**, lo que indica que el desempeño previo del estudiante es el principal indicador del resultado final.  
La variable **Faltas** presenta una tendencia negativa, sugiriendo que una mayor cantidad de ausencias puede afectar el rendimiento académico. En general, los coeficientes obtenidos son coherentes con el contexto educativo y permiten una interpretación clara del modelo.


#### 8. Discusión 

A pesar de ser un modelo lineal simple, la regresión múltiple mostró un desempeño sólido para explicar la variabilidad de las notas finales. La fuerte relación entre evaluaciones parciales y resultado final evidencia que el historial académico reciente es un factor clave.  

Sin embargo, existen algunas limitaciones importantes: posible colinealidad entre G1 y G2, la suposición de relaciones estrictamente lineales y la ausencia de variables contextuales adicionales. Como mejora futura, podrían explorarse modelos no lineales o técnicas de regularización para comparar el desempeño y reducir posibles redundancias.



# 6. Reflexión y conclusiones
El proyecto mostró que la preparación de los datos es esencial para el rendimiento del modelo. La limpieza de outliers y el tratamiento de variables categóricas mejoraron la precisión, destacando G2 como el principal predictor de la nota final. La selección adecuada de características redujo la colinealidad y resultó en un modelo simple e interpretable. Como limitación, el modelo lineal no capta caídas bruscas de desempeño causadas por factores externos no registrados.